In [42]:
import pandas
import math
import numpy as np

def get_res(DATASET, SKEW, PARTIES):
    experiment_directory = f"{DATASET}_non-IID_qty_skew/{PARTIES}_parties/"

    distributions = []
    with open(f"{experiment_directory}individual/{SKEW}/{DATASET.lower()}_qty_skew_{SKEW}_{PARTIES}clients_distribution.txt", "r") as reader:
        for i in range(PARTIES):
            distributions.append(reader.readline())

    weighted_mean = {'client_lr':0.0,
                    'client_momentum':0.0,
                    'batch_size':0}
    
    ratios = np.zeros(PARTIES, dtype=float)
    lr = np.zeros(PARTIES, dtype=float)
    momentum = np.zeros(PARTIES, dtype=float)
    batch_size = np.zeros(PARTIES, dtype=float)
    accuracy = np.zeros(PARTIES, dtype=float)

    # Get individual data and samples ratio
    for i in range(PARTIES):

        client_result = ""

        try:
            client_result = pandas.read_csv(f"{experiment_directory}individual/{SKEW}/{SKEW}_qty_{PARTIES}clts_clt{i}.txt")
            #display(client_result.head(3).get(['val_accuracy','client_lr','client_momentum','round_epochs','batch_size']))
            #print(distributions[i])

            # get ratio
            ratio = float(distributions[i].split(',')[2].replace(' percentage : ',''))
            
            client_lr = client_result.head(1).get(['client_lr']).values[0][0]
            client_momentum = client_result.head(1).get(['client_momentum']).values[0][0]
            client_batch_size = client_result.head(1).get(['batch_size']).values[0][0]
            client_accuracy = client_result.head(1).get(['val_accuracy']).values[0][0]
            
            ratios[i] = ratio
            lr[i] = client_lr
            momentum[i] = client_momentum
            batch_size[i] = client_batch_size
            accuracy[i] = client_accuracy
                        
        except:
            print(f"File does not exist, client {i} crashed.")
    
    best_acc = np.max(accuracy)
    
    for i in range(PARTIES):
        #################  AGGREGATION METHOD BEGIN  #################
        weighted_mean['client_lr'] += ratios[i] * lr[i] * (1-lr[i]) * (1-abs(best_acc-accuracy[i])) / 10
        weighted_mean['client_momentum'] += min(1.0/PARTIES ,ratios[i] * momentum[i] * (1-momentum[i]) * (1-abs(best_acc-accuracy[i])) * 10)
        weighted_mean['batch_size'] += ratios[i] * batch_size[i] * accuracy[i]
        #################  AGGREGATION METHOD END  #################
        
    # Get fedavg data
    try:
        with open(f"{experiment_directory}fedavg/{DATASET.lower()}_qty_skew_{SKEW}_{PARTIES}clients.txt", "r") as reader:
            print("Federated average best parameters\n")

            for i in range(1):
                print(reader.readline())

    except:
        print("File does not exist")

    print(weighted_mean)
    
    # Get fedavg intervals search
    try:
        with open(f"{experiment_directory}intervals/intervals_{DATASET.lower()}_{SKEW_TYPE}_skew_{SKEW}_{PARTIES}clients.txt", "r") as reader:
            print("\nFederated average best intervals\n")

            for i in range(5):
                print(reader.readline())

    except:
        print("File does not exist")
    
DATASET= "CIFAR10" #["MNIST", "EMNIST", "SVHN_CROPPED", "CIFAR10"]
SKEW="5.0" #["0.1", "0.4", "1.0", "2.0", "5.0"] for qty skew
PARTIES=20 #["10", "20"]
get_res(DATASET, SKEW, PARTIES)

Federated average best parameters

(0.38380003, {'act_fn': <jax.custom_derivatives.custom_jvp object at 0x7ff777a31c88>, 'client_lr': 0.05, 'server_lr': 0.03162277660168379, 'client_momentum': 0.6, 'server_momentum': 0.9, 'batch_size': 16, 'epochs_per_round': 2, 'rounds': 30})

{'client_lr': 0.011834833770706828, 'client_momentum': 0.7277473933291141, 'batch_size': 12.217965464200974}
File does not exist
